# ICT for Development - Collaborative API library

## Overview

This notebook aims to compile beginner-friendly documentation of APis relevant to the work of civic actors active in the field of international development. The project was started in 2024 by [Cedric Lombion](https://civicliteraci.es) to support his teaching at the University of Turin's ICT for Development and Social Good master's programme. Nonetheless, the project bears no formal affiliation with the university. 

Each entry aims to document key information and examples about the API being documented, with a focus on making them more accessible to beginners to APIs and command line scripting.

We recommend viewing this notebook from the [nbsanity viewer](https://nbsanity.com/static/5cde5825c56c8bc90bf503ed1e340ca0/api_testbench.html) for better navigation.

## Contributing

### Main Steps

1. Fork the repository
2. add your API description and examples
3. Ensure that your examples are well documented an suitable for beginners
4. send a pull request which will be reviewed for addition.

### Documentation Rules

1. All description items should be filled. 
2. The "use cases" descriptor should include examples relevant to international development projects
3. Up to 5 examples should be provided, each with a small introduction, with a focus on the examples relevant to international development

| Property                | Description                                                        | Possible Values |
|-------------------------|--------------------------------------------------------------------|-----------------|
| Main Website            | The primary website for the API                                    |                 |
| API Documentation       | Links to detailed API documentation                                |                 |
| Purpose                 | The main purpose or functionality of the API                       |                 |
| Example Use Cases       | Practical examples of how the API can be used                      |                 |
| Free API Calls          | Whether the API offers free calls                                  |Yes with API / Yes without API|
| Rate Limits for Free Usage | The rate limit for free API usage                               |                 |
| Other Limitations       | Other potential limitations not covered by the free usage details  |                 |



### Script writing

1. The code should be commented and structured for readbility
2. curl with --data-urlencode is the expected command for all queries. The use of alternatives should be explained.
3. If the API example output is too long, use the ```jq_outliner``` variable to produce a summarized output.

For suggestions or improvements, open an issue on the repository. Alternatively, contact us at contact [at] civicliteraci.es

In [35]:
# outliner function to generate informative summaries of verbose API outputs. 
# note that the order of the top level json objects will not be preserved

jq_outline='
# Collect all unique paths as arrays of segments
[ path(..) | map(if type == "number" then "[]" else tostring end) | join(".") ] | unique | map(split("."))

# Group paths by the first segment, filtering out any that do not have a second segment
| group_by(.[0]) | map(select(.[0][0] != null))

# Convert groups to objects with proper handling of potentially null keys
| map({
  key: (.[0][0] // "undefined_key"),  # Use 'undefined_key' or another placeholder if the key is null
  value: (map(.[1:]) | map(join(".")) | map(select(length > 0) | ".\(.)") | unique)
})

# Merge all objects into a single object, handling null keys properly
| from_entries
'

## Genderize API
* **main website**: https://genderize.io
* **API docs**: https://genderize.io/documentation
* **Purpose:** predicts the gender of an individual based on their name and (optionally) country
* **Example use cases:**
    * Disaggregate past research data by gender
    * Evaluate potential bias in projects that do not collect gender information
* **Free API calls**: Yes, without API key
* **Rate limiting for free usage**: 100 names/day & 10 names per batch request
* **Other Limitations:** 
    * binary gender model
    * using localization on a batch request will apply the given country_id to every name in the batch


### Basic example

In [8]:
# We use the option --data-urlencode to make it easy to include names with special characters
curl -G "https://api.genderize.io/" --data-urlencode "name=Marie Anne" --data-urlencode "country=FR"

{"count":481613,"name":"Marie Anne","gender":"female","probability":1.0}


### Multiple names in a request

A loop can be used to construct a request that includes multiple names drawn from an array.  
the API accepts only up to 10 names.  
Adding a localization will apply to all names in the request.

In [13]:
# Defining our arrays

name_list=("yamen" "lawson" "maria" "alejandro")
country=("sy" "tg" "it" "es")

# Start building the curl command
request="curl -s -G \"https://api.genderize.io?\""

# Add each name to the curl command with --data-urlencode
for name in "${name_list[@]}"
do
    request+=" --data-urlencode \"name[]=${name}\""
done

# Check that the command is properly formed
echo "$request"

# Excute the command
bash -c "$request" | jq

curl -s -G "https://api.genderize.io?" --data-urlencode "name[]=yamen" --data-urlencode "name[]=lawson" --data-urlencode "name[]=maria" --data-urlencode "name[]=alejandro"
[
  {
    "count": 814,
    "name": "lawson",
    "gender": "male",
    "probability": 0.88
  },
  {
    "count": 2687884,
    "name": "maria",
    "gender": "female",
    "probability": 1
  },
  {
    "count": 386853,
    "name": "alejandro",
    "gender": "male",
    "probability": 1
  }
]


### Automate name and country requests and CSV export

We can create a loop that makes it possible to automate requests with both the name and country included.  
We will process the returned json with jq to transform the results into tabular data.

In [14]:
# we declare our arrays
name_list=("yamen" "lawson" "maria" "alejandro")
country=("sy" "tg" "it" "es")

# we count and store the number of values in our array to reuse it in the loop
length=${#country[@]}

# we loop through the arrays
for ((i=0; i<length; i++))
do
    curl -s -G "https://api.genderize.io/" \
    --data-urlencode "name=${name_list[$i]}" \
    --data-urlencode "country_id=${country[$i]}" |
    jq '[.name, .gender] | @csv'
done

"\"yamen\",\"male\""
"\"lawson\",\"male\""
"\"maria\",\"female\""
"\"alejandro\",\"male\""


## SerpAPI
* **main website**: https://serpapi.com/
* **API docs**: 
    * https://serpapi.com/
    * https://serpapi.com/playground
    * https://serpapi.com/search-api
* **Purpose**: getting JSON formatted search results from multiple search engines
* **Example use cases**:
    * Monitoring the news on specific topics to improve organisational response in areas with no field staff
    * Monitoring the impact of campaign communication efforts
    * Monitor Google News for reporting on corruption cases
    * Generate a list of amenities (e.g. doctor office) in an area to evaluate accessibility
* **Free API calls**: Yes, with API key
* **Rate limiting for free usage**: 100 calls / month
* **Other Limitations:** 

### Simple query filtered with jq

Example of a basic Google search query, as seen on their front page (only query that resolves without an API key).  
The output is summarized with the outliner function.

In [49]:
curl -s "https://serpapi.com/search.json" \
  --get \
  --data-urlencode "q=Coffee" \
  --data-urlencode "location=Austin, Texas, United States" \
  --data-urlencode "hl=en" \
  --data-urlencode "gl=us" \
  --data-urlencode "google_domain=google.com" \
  | jq "$jq_outline"

{
  "immersive_products": [
    ".[]",
    ".[].extensions",
    ".[].extensions.[]",
    ".[].extracted_original_price",
    ".[].extracted_price",
    ".[].immersive_product_page_token",
    ".[].original_price",
    ".[].price",
    ".[].serpapi_link",
    ".[].source",
    ".[].thumbnail",
    ".[].title"
  ],
  "local_map": [
    ".gps_coordinates",
    ".gps_coordinates.altitude",
    ".gps_coordinates.latitude",
    ".gps_coordinates.longitude",
    ".image",
    ".link"
  ],
  "local_results": [
    ".more_locations_link",
    ".places",
    ".places.[]",
    ".places.[].address",
    ".places.[].description",
    ".places.[].lsig",
    ".places.[].place_id",
    ".places.[].place_id_search",
    ".places.[].position",
    ".places.[].rating",
    ".places.[].reviews",
    ".places.[].reviews_original",
    ".places.[].service_options",
    ".places.[].service_options.dine_in",
    ".places.[].thumbnail",
    ".places.[].title",
    ".places.[].type"
  ],
  "organic_results": [

### Filtered data and export to CSV

Same basic query, but with an explicit attribute filter and csv conversion with jq

In [39]:
curl -s "https://serpapi.com/search.json" \
  --get \
  --data-urlencode "q=Coffee" \
  --data-urlencode "location=Austin, Texas, United States" \
  --data-urlencode "hl=en" \
  --data-urlencode "gl=us" \
  --data-urlencode "google_domain=google.com"\
  | jq '.organic_results[] | [.title,.link, .snippet] | @csv'

"\"Coffee\",\"https://en.wikipedia.org/wiki/Coffee\",\"Coffee is a beverage brewed from roasted coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans, primarily due ...\""
"\"Starbucks Coffee Company\",\"https://www.starbucks.com/\",\"More than just great coffee. Explore the menu, sign up for Starbucks® Rewards, manage your gift card and more.\""
"\"Coffee | Origin, Types, Uses, History, & Facts\",\"https://www.britannica.com/topic/coffee\",\"Coffee, beverage brewed from the roasted and ground seeds of the tropical evergreen coffee plant of African origin. Coffee is one of the ...\""
"\"What is Coffee?\",\"https://www.ncausa.org/About-Coffee/What-is-Coffee\",\"Coffee traces its origin to a genus of plants known as Coffea. Within the genus there are over 500 genera and 6,000 species of tropical trees and shrubs.\""
"\"Coffee | The Nutrition Source - hsph.harvard.edu\",\"https://www.hsph.harvard.edu/nutritionsource/food-features/coffee/\",\"

## Country Info API

* **main website**: https://countryinfoapi.com
* **API docs**: https://countryinfoapi.com
* **Purpose**: fetch additional info about a country/continent etc
* **Example use cases**: 
    * adding contextual geographical information to a list of cities or countries
* **Free API calls**: Yes, without API key
* **Rate limiting for free usage**: unknown
* **Other Limitations:**



### Loop example

Using a loop to get additional info about a list of countries.

In [81]:
# the api structure doesn't use a typical REST format, so we use jq's uri encoding
# instead of --data-urlencode

countries=("France" "Togo" "Italy" "Germany" "Syria" "Ivory Coast" "New Zealand")

for i in "${countries[@]}"
do  
    country=$(echo -n "$i" | jq -sRr @uri)
    curl -s "https://countryinfoapi.com/api/countries/name/$country" | jq '[.name,.region,.population] | @csv'
done


"\"France\",\"Europe\",67391582"
"\"Togo\",\"Africa\",8278737"
"\"Italy\",\"Europe\",59554023"
"\"Germany\",\"Europe\",83240525"
"\"Syria\",\"Asia\",17500657"
"\"Ivory Coast\",\"Africa\",26378275"
"\"New Zealand\",\"Oceania\",5084300"


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Error</title>
</head>
<body>
<pre>Cannot GET /api/population/greater</pre>
</body>
</html>


## Open Meteo API

* **main website**: https://api.open-meteo.com
* **API docs**: https://open-meteo.com/en/docs
* **Purpose**: get historical weather data, weather forecast data, marine weather data and climate change-related data
* **Example use cases**: 
    * monitor flood risks related to heavy rains
    * Monitor weather to adjust field deployments
    * Compare how different locations are affected by climate change
* **Free API calls**: Yes, without API key
* **Rate limiting for free usage**: 10,000 /day
* **Other Limitations:**

### Basic example

Summarized wth the outliner function.

In [58]:
curl -s "https://api.open-meteo.com/v1/forecast" \
  --get \
  --data-urlencode "latitude=52.5200" \
  --data-urlencode "longitude=13.4050" \
  --data-urlencode "hourly=precipitation" \
  --data-urlencode "start_date=2024-04-10" \
  --data-urlencode "end_date=2024-04-11" \
  --data-urlencode "timezone=auto" \
| jq "$jq_outline"

{
  "elevation": [],
  "generationtime_ms": [],
  "hourly": [
    ".precipitation",
    ".precipitation.[]",
    ".time",
    ".time.[]"
  ],
  "hourly_units": [
    ".precipitation",
    ".time"
  ],
  "latitude": [],
  "longitude": [],
  "timezone": [],
  "timezone_abbreviation": [],
  "utc_offset_seconds": []
}


### Calculating daily precipitations

Relevant for analysis around flood risks.  
Same query as before, but the timeframe is extened over 5 days instead of 24h

In [59]:
curl -s "https://api.open-meteo.com/v1/forecast" \
  --get \
  --data-urlencode "latitude=52.5200" \
  --data-urlencode "longitude=13.4050" \
  --data-urlencode "hourly=precipitation" \
  --data-urlencode "start_date=2024-04-10" \
  --data-urlencode "end_date=2024-04-15" \
  --data-urlencode "timezone=auto" \
| jq '[.hourly.precipitation[]] | add'

0.9000000000000001


## Healthsites.io API

* **main website**: 
* **API docs**: 
* **Purpose:** 
* **Limitations:** 

# World Bank API
by Alejandro Sainz-Pardo

* **main website**: [World Bank API](https://documents.worldbank.org/en/publication/documents-reports/api) 
* **API docs**: All details on how to work with the API in the [link](https://documents.worldbank.org/en/publication/documents-reports/api)
* **Purpose:** The World Bank offers an API that allows for the search and retrieval of the public, Bank documents available in the Documents & Reports site.  Records can be retrieved in a format useful for research and for inclusion in web sites outside of Documents & Reports and the World Bank. 
* **Limitations:** Imagination

In [8]:
#What follows is a simple request querying for records of documents related to wind turbines.  It also requests that the list returned be in a JSON format and
# that the country and document date be returned with each record.  (Other key, record fields, like display title and URL, are always returned):

#Bash

#curl -s "https://search.worldbank.org/api/v2/wds?format=json&qterm=wind%20turbine&fl=docdt,count,country"

#Python

!curl -s "https://search.worldbank.org/api/v2/wds?format=json&qterm=wind%20turbine&fl=docdt,count,country"




{"rows":10,"os":0,"page":1,"total":546,"documents":{"D11831032":{"id":"11831032","count":"Mexico","entityids":{"entityid":"000334955_20100222012800"},"docdt":"2006-04-24T00:00:00Z","abstracts":{"cdata!":"The objective of the Wind Umbrella (La\n            Venta II) Project aims to reduce greenhouse gases emissions\n            from power generation in Mexico and promote investment in\n            wind energy in Mexico to diversify the sources of power\n            generation in the country. There two components to the\n            projects. The first component of the project is purchase of\n            carbon emissions reductions. Certified emissions reduction\n            (CFE) uses a long-term power planning simulation model\n            (DECADES) to select future power plants that are needed for\n            the expansion of the country's electricity system. On\n            the basis of such modeling results it is possible to compare\n            alternative scenarios of new generat

## nationalize.io API
Elena Bolla
* **main website**: https://nationalize.io
* **API docs**: https://nationalize.io/documentation
* **Purpose:** predicts the nationality of an individual based on a person's last name
* **Example use cases:**
    * Analyse nationality distribution in a big database
* **Free API calls**: Yes, No sign-up or API key is needed.
* **Rate limiting for free usage**: The API is free for up to 100 names/day.
* **Other Details:** 
    * Diacritics from any language as well as non-latin alphabets.

In [3]:
#Basic Example

curl "https://api.nationalize.io/?name=Lombion"

{"count":2,"name":"Lombion","country":[{"country_id":"GP","probability":0.9266678414988436},{"country_id":"FR","probability":0.07333215850115644}]}


In [2]:
#Trying with multiple surnames

surname_list=("lombion" "bolla" "Zoki+Azata" "bash")
for surname in "${surname_list[@]}"
do
    curl -s "https://api.nationalize.io/?name={$surname}" | jq
done

{
  "count": 2,
  "name": "lombion",
  "country": [
    {
      "country_id": "GP",
      "probability": 0.9266678414988436
    },
    {
      "country_id": "FR",
      "probability": 0.07333215850115644
    }
  ]
}
{
  "count": 74,
  "name": "bolla",
  "country": [
    {
      "country_id": "HU",
      "probability": 0.28831309795980176
    },
    {
      "country_id": "CM",
      "probability": 0.07610102637257624
    },
    {
      "country_id": "IT",
      "probability": 0.06699569544055527
    },
    {
      "country_id": "IN",
      "probability": 0.06603363240470841
    },
    {
      "country_id": "UY",
      "probability": 0.053718371557112635
    }
  ]
}
{
  "count": 8,
  "name": "Zoki Azata",
  "country": [
    {
      "country_id": "GH",
      "probability": 0.22861410870767024
    },
    {
      "country_id": "CI",
      "probability": 0.22861410870767024
    },
    {
      "country_id": "BJ",
      "probability": 0.22861410870767024
    },
    {
      "country_id": "NG",


# UN Data Portal API
By Lucia Canziani 
* **Main website**: https://population.un.org/dataportal 
* **API docs**: https://population.un.org/dataportal/about/dataapi 
* **Purpose**: it provides access to global datasets of demographic indicators produced by the United Nations Population Division
* **Example use cases**: 
  * Areas with higher healthcare demands, such as those with ageing populations or high frequency of certain diseases, can be identified using demographic data, such as age distribution, population density, and the prevalence of particular diseases in different regions.
  * Precise demographic data is essential for preparing and lessening the effects of natural disasters or humanitarian crises in the context of disaster preparedness and response.
  * It can be used to examine trends in migration, urbanisation rates, and population growth in various geographic areas.
* **Free API calls**: Yes, without API key.
* **Rate limiting for free usage**:Unknown
* **Other Limitations**: 
  * Although there is a lot of demographic data available through the UN Population Division API, not all datasets may be accessible through the API.

In [1]:
# Test the API
curl -s https://population.un.org/dataportalapi/api/v1/topics?sort=sortOrder

{"pageNumber":1,"pageSize":100,"previousPage":null,"nextPage":null,"pages":1,"total":10,"data":[{"id":0,"name":"Not applicable","shortName":"NA","sortOrder":0},{"id":1,"name":"Population","shortName":"Pop","sortOrder":1},{"id":2,"name":"Fertility","shortName":"Fert","sortOrder":2},{"id":3,"name":"Mortality","shortName":"Mort","sortOrder":3},{"id":4,"name":"International Migration","shortName":"iMigration","sortOrder":4},{"id":5,"name":"Family Planning","shortName":"FP","sortOrder":5},{"id":6,"name":"Marital Status","shortName":"MarStat","sortOrder":6},{"id":7,"name":"All Components","shortName":"All","sortOrder":7},{"id":8,"name":"Child Mortality","shortName":"IGME","sortOrder":8},{"id":9,"name":"Maternal Mortality","shortName":"MMEIG","sortOrder":9}]}


In [2]:
# Combine the curl with jq to read the information more easily 
curl -s https://population.un.org/dataportalapi/api/v1/topics?sort=sortOrder | jq

{
  "pageNumber": 1,
  "pageSize": 100,
  "previousPage": null,
  "nextPage": null,
  "pages": 1,
  "total": 10,
  "data": [
    {
      "id": 0,
      "name": "Not applicable",
      "shortName": "NA",
      "sortOrder": 0
    },
    {
      "id": 1,
      "name": "Population",
      "shortName": "Pop",
      "sortOrder": 1
    },
    {
      "id": 2,
      "name": "Fertility",
      "shortName": "Fert",
      "sortOrder": 2
    },
    {
      "id": 3,
      "name": "Mortality",
      "shortName": "Mort",
      "sortOrder": 3
    },
    {
      "id": 4,
      "name": "International Migration",
      "shortName": "iMigration",
      "sortOrder": 4
    },
    {
      "id": 5,
      "name": "Family Planning",
      "shortName": "FP",
      "sortOrder": 5
    },
    {
      "id": 6,
      "name": "Marital Status",
      "shortName": "MarStat",
      "sortOrder": 6
    },
    {
      "id": 7,
      "name": "All Components",
      "shortName": "All",
      "sortOrder": 7
    },
    {
      

In [3]:
# Find the indicator ID about the topic of international migration (topic id: 4)
curl -s ID number: https://population.un.org/dataportalapi/api/v1/topics/4/indicators | jq

[
  {
    "topicId": 4,
    "topicName": "International Migration",
    "topicShortName": "iMigration",
    "indicatorId": 66,
    "indicatorName": "Crude rate of net migration",
    "indicatorDisplayName": "Crude rate of net migration",
    "indicatorShortName": "TNetMigRT",
    "indicatorDescription": "The crude rate of net migration is defined as the ratio of net migration during the year to the average population in that year. The value is expressed per 1000 persons.",
    "dimAge": false,
    "dimSex": false,
    "dimVariant": true,
    "dimCategory": false,
    "defaultAgeId": 188,
    "defaultSexId": 3,
    "defaultVariantId": 4,
    "defaultCategoryId": 0,
    "unitScaling": 1,
    "formatString": "### ### ##0",
    "unitLongLabel": "persons",
    "unitShortLabel": "persons",
    "variableType": "absolute",
    "sourceId": 25,
    "sourceName": "World Population Prospects",
    "sourceYear": 2022,
    "sourceStartYear": 1950,
    "sourceEndYear": 2100,
    "sourceCitation": "Un

In [4]:
# Retrieve the location ID of your interest (example: India and Brazil)
curl -s https://population.un.org/dataportalapi/api/v1/locations/IND,BRA?sort=id| jq


[
  {
    "id": 76,
    "name": "Brazil",
    "iso3": "BRA",
    "iso2": "BR",
    "longitude": -51.9252815246582,
    "latitude": -14.235004425048828
  },
  {
    "id": 356,
    "name": "India",
    "iso3": "IND",
    "iso2": "IN",
    "longitude": 78.96288299560547,
    "latitude": 20.59368324279785
  }
]


In [5]:
# Retrieve data about Total net-migration (indicator id 65) in India and Brazil between 1980 and 2023
curl -s https://population.un.org/dataportalapi/api/v1/data/indicators/65,66/locations/76,356/start/1980/end/2023 | jq

{
  "pageNumber": 1,
  "pageSize": 100,
  "previousPage": null,
  "nextPage": "https://population.un.org/dataportalapi/api/v1/data/indicators/65,66/locations/76,356/start/1980/end/2023?pageNumber=2&pageSize=100",
  "pages": 3,
  "total": 248,
  "data": [
    {
      "locationId": 76,
      "location": "Brazil",
      "iso3": "BRA",
      "iso2": "BR",
      "locationTypeId": 4,
      "indicatorId": 65,
      "indicator": "Total net-migration",
      "indicatorDisplayName": "Total net number of migrants",
      "sourceId": 25,
      "source": "World Population Prospects",
      "revision": 0,
      "variantId": 4,
      "variant": "Median",
      "variantShortName": "Median",
      "variantLabel": "Median",
      "timeId": 31,
      "timeLabel": "1980",
      "timeMid": "1980.5",
      "categoryId": 0,
      "category": "Not applicable",
      "estimateTypeId": 1,
      "estimateType": "Model-based Estimates",
      "estimateMethodId": 2,
      "estimateMethod": "Interpolation",
      "

In [6]:
# Create a header for the csv file 
echo "Year,Location,Indicator,Value" > Total_net_migration.csv

# Filter the data I need and extract it into a csv
curl -s https://population.un.org/dataportalapi/api/v1/data/indicators/65/locations/76,356/start/1980/end/2023 | jq -r '.data[] | [.location, .timeLabel, .indicator, .value] | @csv' >> Total_net_migration.csv


In [7]:
# Check the csv has been downloaded correctly
cat Total_net_migration.csv

Year,Location,Indicator,Value
"Brazil","1980","Total net-migration",11959
"Brazil","1981","Total net-migration",-14927
"Brazil","1982","Total net-migration",-23807
"Brazil","1983","Total net-migration",-17884
"Brazil","1984","Total net-migration",-20225
"Brazil","1985","Total net-migration",-13535
"Brazil","1986","Total net-migration",-24426
"Brazil","1987","Total net-migration",-22335
"Brazil","1988","Total net-migration",-11309
"Brazil","1989","Total net-migration",-2800
"Brazil","1990","Total net-migration",6457
"Brazil","1991","Total net-migration",-30659
"Brazil","1992","Total net-migration",-20270
"Brazil","1993","Total net-migration",-21956
"Brazil","1994","Total net-migration",-11642
"Brazil","1995","Total net-migration",-26337
"Brazil","1996","Total net-migration",-24330
"Brazil","1997","Total net-migration",-20429
"Brazil","1998","Total net-migration",-22409
"Brazil","1999","Total net-migration",-26152
"Brazil","2000","Total net-migration",-1761
"Brazil","2001","Total net-mig